# How to use Flask-sqlalchemy db.Model derived classes in JupyterLab

20201111 : 

Web applications developed using Flask and Flask-sqlalchemy use a declarative base class db.Model to implement SQLAlchemy. I often want to develop and test classes derived from db.Model in JupyterLab.  This notebook gives a pattern showing how to import the classes into JupyterLab, create and push the Flask app, and connect to multiple databases using the Flask db = SQLAlchemy() instance.

This example uses sqlite but the pattern is applicable to other databases.  This notebook connects to three simple and arbitrary databases.

## Import Flask app, SqlAlchemy instance, and database table classes

In [1]:
# Input app directory and database filename
run_config = 'development' # 'development' or 'production'
app_directory = 'jupyterlab_flask_sqlalchemy'
app_path = '/Users/Phil/Documents/webapps/' + app_directory

In [2]:
# Imports
import sys

from platform import python_version
print(f"Python version : {python_version()}")

if app_path not in sys.path:
    sys.path.append(app_path)
    
from app import db, create_app
from app.models_chinook import Album, Artist
from app.models_db2 import Company, Country, Employee
from app.models_db3 import Pet, User, Post

Python version : 3.9.0


In [3]:
# create flask app which creates sqlalchemy object 'db'
current_app = create_app(run_config)
current_app.app_context().push()

PAB> Python version 3.9.0 (v3.9.0:9cf6752276, Oct  5 2020, 11:29:23) 
[Clang 6.0 (clang-600.0.57)]
PAB> Run configuration = development


## Examples using Flask-sqlalchemy db object

In [4]:
print(f"db.session is : {db.session}")
print(f"\nDefault database is : {db.get_engine(bind=None)}")
print(f"\nDB2 is : {db.get_engine(bind='db2')}")
print(f"\nDB3 is : {db.get_engine(bind='db3')}")

db.session is : <sqlalchemy.orm.scoping.scoped_session object at 0x7fe328777310>

Default database is : Engine(sqlite:////Users/Phil/Documents/webapps/jupyterlab_flask_sqlalchemy/chinook.sqlite)

DB2 is : Engine(sqlite:////Users/Phil/Documents/webapps/jupyterlab_flask_sqlalchemy/jfs1.sqlite)

DB3 is : Engine(sqlite:////Users/Phil/Documents/webapps/jupyterlab_flask_sqlalchemy/jfs2.sqlite)


In [5]:
db.get_engine(bind=None).table_names()

['album', 'artist']

In [6]:
db.get_engine(bind='db2').table_names()

['company', 'country', 'employee']

In [7]:
db.get_engine(bind='db3').table_names()

['pet', 'post', 'user']

## Sample sqlalchemy queries

### Query the default database (Chinook)

In [8]:
q = db.session.query(Album).first()
q.title

'For Those About To Rock We Salute You'

In [9]:
q = db.session.query(Album).filter(Album.id==1).first()
q.title

'For Those About To Rock We Salute You'

In [10]:
# Get all records for artistid
q = db.session.query(Album).filter(Album.artistid==1).all()
for item in q:
    print(item.id, '\t', item.artistid, '\t', item.title)

1 	 1 	 For Those About To Rock We Salute You
4 	 1 	 Let There Be Rock


In [11]:
q = db.session.query(Album, Artist).filter(Album.artistid==1).filter(Artist.id==1)
for item in q:
    print(item.Album.id, '\t', item.Album.artistid, '\t', item.Artist.name, '\t', item.Album.title)

1 	 1 	 AC/DC 	 For Those About To Rock We Salute You
4 	 1 	 AC/DC 	 Let There Be Rock


In [12]:
q = db.session.query(Album, Artist).filter(Artist.name=='AC/DC').filter(Album.artistid==Artist.id)
for item in q:
    print(item.Album.id, '\t', item.Album.artistid, '\t', item.Artist.name, '\t', item.Album.title)

1 	 1 	 AC/DC 	 For Those About To Rock We Salute You
4 	 1 	 AC/DC 	 Let There Be Rock


### Query database 2 (DB2)

In [13]:
q = db.session.query(Employee).all()
for item in q:
    print(item.id, '\t', item.last_name, '\t', item.company_id)

1 	 Adams 	 1
2 	 Edwards 	 2
3 	 Peacock 	 3
4 	 Park 	 1
5 	 Johnson 	 1
6 	 Ramos 	 10


In [14]:
q = db.session.query(Company).all()
for item in q:
    print(item.id, '\t', item.name)

1 	 Samsung Electronics
2 	 Apple Inc.
3 	 Foxconn
4 	 HP
5 	 IBM
6 	 Amazon.com
7 	 Microsoft
8 	 Sony
9 	 Panasonic
10 	 Google
12 	 Toshiba
13 	 LG Electronics
14 	 Intel


In [15]:
q = db.session.query(Company, Employee).filter(Company.name=='Samsung Electronics').filter(Company.id==Employee.company_id)
for item in q:
    print(item.Company.id, '\t', item.Company.name, '\t', item.Employee.last_name)

1 	 Samsung Electronics 	 Adams
1 	 Samsung Electronics 	 Park
1 	 Samsung Electronics 	 Johnson


### Query database 3 (DB3)

In [16]:
q = db.session.query(Pet)
for item in q:
    print(item.name, '\t', item.animal, '\t', item.owner_id)

Baron 	 dog 	 3
Auzzie 	 dog 	 2


In [17]:
q = db.session.query(Pet, User).filter(Pet.name=='Baron').filter(User.id==Pet.owner_id)
for item in q:
    print(item.Pet.name, '\t', item.Pet.animal, '\t', item.Pet.owner_id, '\t', item.User.name)

Baron 	 dog 	 3 	 Laura Callahan


## Close sqlalchemy session

In [18]:
db.session.close_all()